In [ ]:
!gdown 1FGul56hum7JLk3WtotNjmzPs9-7yajOS
!rm *.csv
!unzip *.zip
!rm *.zip

Downloading...
From: https://drive.google.com/uc?id=1FGul56hum7JLk3WtotNjmzPs9-7yajOS
To: /content/ml-usi-disease-ds-clean.zip
100% 77.7k/77.7k [00:00<00:00, 70.3MB/s]
Archive:  ml-usi-disease-ds-clean.zip
  inflating: thyroid-clean.csv       
  inflating: diabetes-clean.csv      
  inflating: heartdis-clean.csv      


In [ ]:
diabetes_codes = {
  "Regular insulin dose": 33,
  "NPH insulin dose": 34,
  "UltraLente insulin dose": 35,
  "Unspecified blood glucose measurement": 48,
  "Pre-breakfast blood glucose measurement": 58,
  "Post-breakfast blood glucose measurement": 59,
  "Pre-lunch blood glucose measurement": 60,
  "Post-lunch blood glucose measurement": 61,
  "Pre-supper blood glucose measurement": 62,
  "Post-supper blood glucose measurement": 63,
  "Pre-snack blood glucose measurement": 64,
  "Hypoglycemic symptoms": 65,
  "Typical meal ingestion": 66,
  "More-than-usual meal ingestion": 67,
  "Less-than-usual meal ingestion": 68,
  "Typical exercise activity": 69,
  "More-than-usual exercise activity": 70,
  "Less-than-usual exercise activity": 71,
  "Unspecified special event": 72
}

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.preprocessing import StandardScaler

# Loading Data

In [ ]:
df_diabetes = pd.read_csv("diabetes-clean.csv")
df_heartdis = pd.read_csv("heartdis-clean.csv")
df_thyroid = pd.read_csv("thyroid-clean.csv")

In [ ]:
df_diabetes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29257 entries, 0 to 29256
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   code           29257 non-null  int64
 1   diabetes_type  29257 non-null  int64
dtypes: int64(2)
memory usage: 457.3 KB


In [ ]:
df_heartdis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 740 entries, 0 to 739
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   age            740 non-null    int64  
 1   sex            740 non-null    object 
 2   cp             740 non-null    object 
 3   trestbps       740 non-null    float64
 4   chol           740 non-null    float64
 5   fbs            740 non-null    bool   
 6   restecg        740 non-null    object 
 7   thalch         740 non-null    float64
 8   exang          740 non-null    bool   
 9   oldpeak        740 non-null    float64
 10  heartdis_type  740 non-null    int64  
dtypes: bool(2), float64(4), int64(2), object(3)
memory usage: 53.6+ KB


In [ ]:
df_thyroid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3772 entries, 0 to 3771
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   age                        3772 non-null   int64  
 1   sex                        3772 non-null   object 
 2   on thyroxine               3772 non-null   bool   
 3   query on thyroxine         3772 non-null   bool   
 4   on antithyroid medication  3772 non-null   bool   
 5   sick                       3772 non-null   bool   
 6   pregnant                   3772 non-null   bool   
 7   thyroid surgery            3772 non-null   bool   
 8   I131 treatment             3772 non-null   bool   
 9   query hypothyroid          3772 non-null   bool   
 10  query hyperthyroid         3772 non-null   bool   
 11  lithium                    3772 non-null   bool   
 12  goitre                     3772 non-null   bool   
 13  tumor                      3772 non-null   bool 

# Making Model (SVM)

## Diabetes Model

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Separate features and target
X = df_diabetes[['code']]
y = df_diabetes['diabetes_type']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the SVM model
diabetes_model = RandomForestClassifier(random_state=42)
diabetes_model.fit(X_train, y_train)

# Make predictions
y_pred = diabetes_model.predict(X_test)
prob = diabetes_model.predict_proba(X_test)

# Calculate metrics
f1 = f1_score(y_test, y_pred, average='weighted')
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')

# Print metrics
print("F1 Score:", f1)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)

F1 Score: 0.7169263760694449
Accuracy: 0.7445317840054683
Precision: 0.7431638393945541
Recall: 0.7445317840054683


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Heart Disease Model

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC

X = df_heartdis.drop('heartdis_type', axis=1)
y = df_heartdis['heartdis_type']

categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
bool_cols = X.select_dtypes(include=['bool']).columns.tolist()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numerical_cols),
        ('cat', OneHotEncoder(), categorical_cols),
        ('scaler', StandardScaler(), numerical_cols)
    ])

heartdis_model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', SVC(probability=True, kernel='rbf', C=10000, random_state=42))
])

heartdis_model_pipeline.fit(X_train, y_train)

y_pred = heartdis_model_pipeline.predict(X_test)
prob = heartdis_model_pipeline.predict_proba(X_test)

f1 = f1_score(y_test, y_pred, average='weighted')
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')

print("F1 Score:", f1)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)

F1 Score: 0.5237860468123625
Accuracy: 0.5608108108108109
Precision: 0.4966118533397394
Recall: 0.5608108108108109


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Thyroid Model

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC

X = df_thyroid.drop('has_thyroid_disease', axis=1)
y = df_thyroid['has_thyroid_disease']

categorical_cols = ['sex']
bool_cols = X.select_dtypes(include=['bool']).columns.tolist()
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numerical_cols),
        ('cat', OneHotEncoder(), categorical_cols),
        ('scaler', StandardScaler(), numerical_cols)
    ])

thyroid_model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('svm', SVC(probability=True, kernel='rbf', C=10, random_state=42))
])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

thyroid_model_pipeline.fit(X_train, y_train)

y_pred = thyroid_model_pipeline.predict(X_test)
prob = thyroid_model_pipeline.predict_proba(X_test)

f1 = f1_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print("F1 Score:", f1)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)

F1 Score: 0.9738884968242767
Accuracy: 0.9509933774834437
Precision: 0.9583333333333334
Recall: 0.9899569583931134


# Fuzzy Inference System

## Taking Input

In [ ]:
def get_user_data(str):
    """
    Arguments
    - str: 'diabetes', 'heartdis', or 'thyroid'
    """
    if str == "diabetes":
        X = df_diabetes.drop('diabetes_type', axis=1)
        y = df_diabetes['diabetes_type']
    elif str == "heartdis":
        X = df_heartdis.drop('heartdis_type', axis=1)
        y = df_heartdis['heartdis_type']
    elif str == "thyroid":
        X = df_thyroid.drop('has_thyroid_disease', axis=1)
        y = df_thyroid['has_thyroid_disease']
    else:
        raise ValueError("Invalid input: expected 'diabetes', 'heartdis', or 'thyroid'")
    _, X_test, _, _ = train_test_split(X, y, test_size=0.2, random_state=42)
    random_selection = X_test.sample(1)
    return random_selection

In [ ]:
def get_dscore(row):
    classes = diabetes_model.classes_
    prediction = diabetes_model.predict_proba(row)
    probas = { classes[i]: prediction[0][i] for i in range(len(classes)) }
    return max(probas.values())


def get_hscore(row):
    classes = heartdis_model_pipeline.classes_
    prediction = heartdis_model_pipeline.predict_proba(row)
    probas = { classes[i]: prediction[0][i] for i in range(len(classes)) }
    return sum(probas[class_label] * int(class_label) for class_label in probas)


def get_tscore(row):
    classes = thyroid_model_pipeline.classes_
    prediction = thyroid_model_pipeline.predict_proba(row)
    probas = { classes[i]: prediction[0][i] for i in range(len(classes)) }
    wtscore = sum(probas[class_label] * int(class_label) for class_label in probas)
    return wtscore / (sum([ int(class_label) for class_label in probas ]))

## Evaluation

In [ ]:
def low(x):
    return max(0, min(1, (0.4 - x) / 0.4))

def moderate(x):
    return max(0, min((x - 0.3) / 0.4, (0.7 - x) / 0.4))

def high(x):
    return max(0, min((x - 0.6) / 0.4, 1))


def fuzzify(score):
    return {
        "low": low(score),
        "moderate": moderate(score),
        "high": high(score)
    }


def evaluate_rules(d_score, h_score, t_score):
    # Fuzzify each input
    d_fuzzy = fuzzify(d_score)
    h_fuzzy = fuzzify(h_score)
    t_fuzzy = fuzzify(t_score)

    # Initialize outputs for risk levels
    low_risk, moderate_risk, high_risk = 0, 0, 0

    # Rules
    # Rule 1: IF d_score is high OR h_score is high THEN risk is high
    high_risk = max(high_risk, max(d_fuzzy['high'], h_fuzzy['high']))
    # Rule 2: IF d_score is moderate AND h_score is moderate AND t_score is moderate THEN risk is moderate
    moderate_risk = max(moderate_risk, min(d_fuzzy['moderate'], h_fuzzy['moderate'], t_fuzzy['moderate']))
    # Rule 3: IF d_score is low AND h_score is low AND t_score is low THEN risk is low
    low_risk = max(low_risk, min(d_fuzzy['low'], h_fuzzy['low'], t_fuzzy['low']))
    # Rule 4: IF d_score is high OR t_score is high THEN risk is moderate
    moderate_risk = max(moderate_risk, max(d_fuzzy['high'], t_fuzzy['high']))

    return low_risk, moderate_risk, high_risk


def defuzzify(low_risk, moderate_risk, high_risk):
    risk_levels = {
        "low": 0.25,
        "moderate": 0.5,
        "high": 0.75
    }
    numerator = (low_risk * risk_levels["low"] +
                 moderate_risk * risk_levels["moderate"] +
                 high_risk * risk_levels["high"])
    denominator = low_risk + moderate_risk + high_risk
    return numerator / denominator if denominator != 0 else 0


def calculate_final_risk(d_score, h_score, t_score, threshold=0.5):
    low_risk, moderate_risk, high_risk = evaluate_rules(d_score, h_score, t_score)
    final_risk_score = defuzzify(low_risk, moderate_risk, high_risk)
    risk_status = "In Danger" if final_risk_score > threshold else "Not in Danger"
    return final_risk_score, risk_status


d_score = get_dscore(get_user_data('diabetes'))
h_score = get_hscore(get_user_data('heartdis'))
t_score = get_tscore(get_user_data('thyroid'))

print("Diabetes Score:", d_score)
print("Heart Disease Score:", h_score)
print("Thyroid Score:", t_score)


final_score, status = calculate_final_risk(d_score, h_score, t_score)
print("Final Risk Score:", final_score)
print("Risk Status:", status)

Diabetes Score: 0.45546908340759445
Heart Disease Score: 0.7107695324795081
Thyroid Score: 0.6256012504488321
Final Risk Score: 0.7030668338571375
Risk Status: In Danger
